---
toc: true
comments: true
layout: post
title: Blog
description: My final blog explaining what i've learned throughout the trimester as well as how i've grown in terms of CSP and all of my findings
type: hacks
courses: { compsci: {week: 7} }
---

Stocks brainstorming
Thought process:

1. **Accessibility and Convenience:**
   - People can access real-time stock information on their smartphones, making it convenient to track their investments on the go.
   - The app can provide a user-friendly interface that simplifies complex financial data, making it easier for the general public to understand and engage with the stock market.

2. **Educational Value:**
   - The app can offer educational resources, tutorials, and articles about investing, helping to increase financial literacy among the general public.
   - Users can learn about investment strategies, market trends, and financial planning, thereby empowering them to make informed investment decisions.

3. **Informed Decision-Making:**
   - The app can provide users with access to comprehensive market analysis and research tools, enabling them to make well-informed decisions about buying, selling, or holding stocks.
   - Real-time alerts and notifications can keep users updated on market changes, ensuring that they can respond promptly to any significant developments.

4. **Democratization of Investing:**
   - By making stock market information easily accessible to the public, the app can help democratize investing, allowing individuals from diverse backgrounds to participate in the financial markets.
   - Lowering the barriers to entry can encourage more people to consider investing as a means of building wealth and securing their financial futures.

5. **Customization and Personalization:**
   - The app can offer personalized recommendations based on users' financial goals, risk tolerance, and investment preferences, helping them build portfolios that align with their specific needs and objectives.
   - Customizable dashboards and watchlists can allow users to track their favorite stocks and monitor specific sectors or industries of interest.

6. **Risk Management and Diversification:**
   - The app can educate users about the importance of diversification and risk management, encouraging them to build diversified portfolios that can mitigate the impact of market volatility.
   - Tools such as portfolio trackers and risk assessment features can assist users in managing and balancing their investment risks effectively.

7. **Transparency and Accountability:**
   - The app can promote transparency in the stock market by providing access to company financial reports, earnings announcements, and corporate governance information, enabling users to make investment decisions based on reliable data.
   - User reviews, ratings, and community forums within the app can foster accountability and promote transparency among companies, encouraging responsible business practices.

8. **Financial Goal Setting and Monitoring:**
   - The app can facilitate financial goal setting and tracking, allowing users to set specific investment targets and monitor their progress over time.
   - Features such as performance trackers, investment calculators, and goal-based savings tools can help users stay focused on their long-term financial objectives.

In conclusion, a well-designed stocks app can be a valuable tool for the public, providing accessibility, education, informed decision-making, democratization of investing, customization, risk management, transparency, and goal-oriented financial planning. By empowering users with comprehensive market insights and resources, such an app can contribute to a more financially literate and empowered society.

<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Loan Calculator</title>
  <link rel="stylesheet" href="styles.css">
</head>
<body>
  <div class="container">
    <h2>Loan Calculator</h2>
    <form id="loan-form">
      <div>
        <label for="amount">Loan Amount:</label>
        <input type="number" id="amount" required>
      </div>
      <div>
        <label for="interest">Interest Rate (in percentage):</label>
        <input type="number" id="interest" step="0.1" required>
      </div>
      <div>
        <label for="years">Loan Duration (in years):</label>
        <input type="number" id="years" required>
      </div>
      <div>
        <button type="submit" class="btn">Calculate</button>
      </div>
    </form>
    <div id="results" class="hidden">
      <h3>Results</h3>
      <p id="monthly-payment"></p>
      <p id="total-payment"></p>
      <p id="total-interest"></p>
    </div>
  </div>
  <script src="script.js"></script>
</body>
</html>


## Note:
This was a code I made in the early parts of the project in markdown to show a potential topic for our finance project. Though in the early works I believe that this helped us understand our code would be better off if we implemented stocks rather than banking and loan calculators as this can often be achieved from any other website. 

from flask import Blueprint, jsonify, request
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime
from flask_restful import Api, Resource
import numpy as np
from flask_cors import CORS
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

stock_api = Blueprint('stock_api', __name__, url_prefix='/api/stocks')
api = Api(stock_api)

CORS(stock_api)


def get_stock_graph(stock_name):
    end_date = datetime.now()
    start_date = end_date - pd.Timedelta(days=4856)

    df = yf.download(stock_name, start=start_date, end=end_date)

    graph = go.Figure(data=go.Candlestick(x=df.index,
                                          open=df['Open'],
                                          high=df['High'],
                                          low=df['Low'],
                                          close=df['Close'],
                                          name=stock_name))
    graph.update_layout(title=f'{stock_name} Stock Price',
                        xaxis_title='Date',
                        yaxis_title='Price')

    graph_data = graph.to_dict()
    graph_data['data'][0]['x'] = graph_data['data'][0]['x'].astype(
        str).tolist()
    graph_data['data'][0]['open'] = graph_data['data'][0]['open'].tolist()
    graph_data['data'][0]['high'] = graph_data['data'][0]['high'].tolist()
    graph_data['data'][0]['low'] = graph_data['data'][0]['low'].tolist()
    graph_data['data'][0]['close'] = graph_data['data'][0]['close'].tolist()

    return graph_data





class _ReadStockGraph(Resource):
    def get(self, stock_name):
        graph = get_stock_graph(stock_name)
        return graph


class _GetLatestStockData(Resource):
    def get(self, stock_name):
        try:

            stock = yf.Ticker(stock_name)
            latest_data = stock.history(period="1d")

            if latest_data.empty:
                return jsonify({'error': 'No data found for the provided stock ticker.'}), 404

            latest_data = latest_data[['Open', 'High', 'Low', 'Close']]
            latest_data = latest_data.rename(
                columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close'})

            latest_data_dict = latest_data.to_dict(orient='records')[0]

            for key, value in latest_data_dict.items():
                if isinstance(value, (float, int)):
                    latest_data_dict[key] = round(value, 2)

            return jsonify(latest_data_dict)
        except Exception as e:
            return jsonify({'error': str(e)}), 500


api.add_resource(_GetLatestStockData, '/latest_data/<string:stock_name>')

api.add_resource(_ReadStockGraph, '/stock_graph/<string:stock_name>')


def train_stock_prediction_model(stock_name):
    end_date = datetime.now()
    start_date = end_date - pd.Timedelta(days=4856)

    df = yf.download(stock_name, start=start_date, end=end_date)

    # preprocess the data
    df['Close'] = df['Close'].pct_change()  # calc daily returns
    df = df.dropna()

    X = df[['Open', 'High', 'Low', 'Close']].values
    y = (df['Close'] > 0).astype(int)  # 1 if price increaseds 0 if decreased

    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(4,)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy',
                  etrics=['accuracy'])
    model.fit(X, y, epochs=10)

    return model


def get_stock_recommendation(stock_name, model):
    end_date = datetime.now()
    start_date = end_date - pd.Timedelta(days=1)

    df = yf.download(stock_name, start=start_date, end=end_date)

    if df.empty:
        return {'error': 'No data found for the provided stock ticker.'}, 404

    df['Close'] = df['Close'].pct_change().iloc[-1]

    X = df[['Open', 'High', 'Low', 'Close']].values
    prediction = model.predict(X)

    recommendation = 'Buy' if prediction > 0.5 else 'Don\'t Buy'
    reason = f'This is based on the current data'

    return {'recommendation': recommendation, 'reason': reason}


class _AnalyzeStock(Resource):
    def get(self, stock_name):
        try:
            model = train_stock_prediction_model(stock_name)
            recommendation = get_stock_recommendation(stock_name, model)
            return jsonify(recommendation)
        except Exception as e:
            return jsonify({'error': str(e)}), 500


api.add_resource(_AnalyzeStock, '/analyze/<string:stock_name>')

In [ ]:
// Function to calculate compound interest
function calculateCompoundInterest(principal, rate, time) {
    // Convert rate to decimal
    rate = rate / 100;
  
    // Calculate compound interest
    let compoundInterest = principal * Math.pow(1 + rate, time) - principal;
  
    return compoundInterest;
}

// Example usage
let principal = 1000; // Example principal amount
let rate = 5; // Example annual interest rate
let time = 3; // Example time period in years

// Call the function and store the result in a variable
let result = calculateCompoundInterest(principal, rate, time);

// Output the result to the console
console.log("The compound interest is: " + result);



# Note:
This is another example of how I attempted to show a possible idea for our code, a little more developed as it is in Javascript rather than HTML allowing it to be transported to our actual site and though it was used for reference, did not make it in the final project. 


Final tweaks and changes to the code:

Main goal 1- Make frontend more appealing to the user. 

Plans:
Set the layout to be well done and add buttons. 
Keep a consise theme throughout the webpage and organize it
Theme should be simple yet tech-y as beginners in tech and finance are our main audience

Main goal 2-reahearse presentation
learn other people's pieces of the project. (easy task for us as we worked through majority of the code as a group)
Delegate who is going to say what, and have one main speaker to present the page.




Peer Reviews:
GROUP StudyBuddy:
HOOK:
Points: 3.6-4.0
Reason: 3.7
Talked about why the project was important and how people use it in everyday life. Explained frontend imagery and why it's relevant.

KNOWLEDGE:
Points: 3.6-4.0
Reason: 3.9

Explained how AI was used to link the backend and front end. AI was connected as a chatbot. Well well-explained and thorough project.

VALUE:
Points: 0.6-1.0
Reason: 0.9
It's well done and I see myself using it in everyday life, it is also user and beginner-friendly.

WOW FACTOR:
Reason:
Frontend was very thorough and well done. The logo was also incredibly thorough and well crafted in a way that's appealing to the user
9.5/10

GROUP Online Groceries:
HOOK:
Points: 3.9
Reason: Explains why the audience would use the project and how it can assist people.
KNOWLEDGE:
Points: 3.9
Reason: Shows different parts of the site and explains how to use it. They also explain how one needs to make an account.

VALUE:
Points: 0.9
Reason: This can be used in everyday life and could be a beneficial extension for online shoppers.

WOW FACTOR:
Reason: The frontend is well done and incredibly appealing. The concept is a good idea as well

GROUP: Cookies
HOOK:
Points: 3.8
Reason: Began the presentation with a hook and got the audience interested in the project. The hook could be a little more elaborate

KNOWLEDGE:
Points: 3.9
Reason: Well done and well thought out. The group thoroughly explains the project and the code used for it. Explains both backend and frontend

VALUE:
Points: 0.7
Reason: It is well thought out but not a largely impactful topic. The project allows viewers to buy cookies.

WOW FACTOR: 0.9
Reason: Agile was glossed over but the frontend was well done and the layout was nice.

Total: 3.8 + 3.9 + 0.7 + 0.9 = 9.3/10

## Tri One Reflection

Reflecting on the first third of the AP CSP (Computer Science Principles) course, I've gained a fundamental understanding of the principles of computing and the role it plays in modern society. The initial modules introduced me to the basics of computational thinking, algorithms, and the broader impact of technology on various aspects of our lives. Understanding the importance of data and its ethical implications has been a crucial part of my learning experience so far. Additionally, the exploration of internet technologies and the significance of cybersecurity has broadened my perspective on the digital world.

In terms of the finance-based projects, they have provided me with valuable insights into the practical applications of computing in the financial sector. Through these projects, I have honed my skills in using programming languages to analyze financial data, assess risks, and make informed decisions. These experiences have underscored the significance of data integrity, security, and privacy in financial transactions, emphasizing the need for robust computational solutions to ensure the smooth functioning of financial systems.

Overall, the combination of theoretical concepts in computer science and their practical applications in finance has deepened my understanding of both fields and highlighted their interdependence in the modern digital landscape. I look forward to further exploring the intricacies of these domains and applying my knowledge to real-world scenarios in the later stages of the AP CSP course.

What I enjoyed-
My favorite part of the first trimester of the course was the freedom with had with design. This allowed me to work at my fullest extent as it allowed me to present something I felt was important or that interested me. This is what helped me in my blog and I enjoyed the freedom given when creating the group passion project which allowed us to do finance, a topic everyone in our group was equally excited about.

What I could do better next tri-
I would enjoy focusing more on using javascript, python and other coding languages that conform witht the AP curriculum as well as practicing theory with pseudo code ratherr than working in HTML, and CSS. I also plan to read about these languages in my free time to expand my personal knowledge.


Reflecting on the initial phase of my AP CSP experience:

During the first few weeks (0-3), I dedicated significant effort to acquainting myself with essential applications like Vscode and Github. Overcoming the learning curve was a bit challenging, but I managed to grasp the basics. Leveraging my prior Python knowledge, I embarked on several projects, one of which involved developing a functional weather program capable of retrieving temperature data based on user input. This project facilitated a solid understanding of JavaScript's frontend capabilities and the integration of external APIs. Additionally, I delved into constructing a spam email classifier, providing an insightful introduction to the realm of AI.

In the subsequent weeks (4-7), my exploration of web programming deepened, focusing on mastering JavaScript syntax, particularly variable usage and looping mechanisms. I also became well-acquainted with the utility of GitHub Utterance comments, recognizing their value in receiving constructive feedback and fostering user engagement. Furthermore, I familiarized myself with fundamental aspects of project management, including team roles and agile methodologies.

The latter phase (weeks 8-12) marked an exciting transition as I enthusiastically contributed to a team project centered around Cryptography. My responsibilities as a backend engineer involved developing AI decryption models. Additionally, I enhanced the project's frontend by incorporating dynamic elements, such as visual effects, and working on the backend through importing different graph versions which were then formatted and displayed. 

Overall, this initial segment of the AP CSP journey has been a rich learning experience, enhancing my proficiency in JavaScript, Python machine learning, and markup languages like HTML. I look forward to leveraging these skills in the forthcoming trimesters to tackle more complex challenges and expand my expertise in the dynamic field of computer science.

## Core commits
from flask import Blueprint, jsonify, request
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime
from flask_restful import Api, Resource
import numpy as np
from flask_cors import CORS
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

stock_api = Blueprint('stock_api', __name__, url_prefix='/api/stocks')
api = Api(stock_api)

CORS(stock_api)


def get_stock_graph(stock_name):
    end_date = datetime.now()
    start_date = end_date - pd.Timedelta(days=4856)

    df = yf.download(stock_name, start=start_date, end=end_date)

    graph = go.Figure(data=go.Candlestick(x=df.index,
                                          open=df['Open'],
                                          high=df['High'],
                                          low=df['Low'],
                                          close=df['Close'],
                                          name=stock_name))
    graph.update_layout(title=f'{stock_name} Stock Price',
                        xaxis_title='Date',
                        yaxis_title='Price')

    graph_data = graph.to_dict()
    graph_data['data'][0]['x'] = graph_data['data'][0]['x'].astype(
        str).tolist()
    graph_data['data'][0]['open'] = graph_data['data'][0]['open'].tolist()
    graph_data['data'][0]['high'] = graph_data['data'][0]['high'].tolist()
    graph_data['data'][0]['low'] = graph_data['data'][0]['low'].tolist()
    graph_data['data'][0]['close'] = graph_data['data'][0]['close'].tolist()

    return graph_data





class _ReadStockGraph(Resource):
    def get(self, stock_name):
        graph = get_stock_graph(stock_name)
        return graph


class _GetLatestStockData(Resource):
    def get(self, stock_name):
        try:

            stock = yf.Ticker(stock_name)
            latest_data = stock.history(period="1d")

            if latest_data.empty:
                return jsonify({'error': 'No data found for the provided stock ticker.'}), 404

            latest_data = latest_data[['Open', 'High', 'Low', 'Close']]
            latest_data = latest_data.rename(
                columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close'})

            latest_data_dict = latest_data.to_dict(orient='records')[0]

            for key, value in latest_data_dict.items():
                if isinstance(value, (float, int)):
                    latest_data_dict[key] = round(value, 2)

            return jsonify(latest_data_dict)
        except Exception as e:
            return jsonify({'error': str(e)}), 500


api.add_resource(_GetLatestStockData, '/latest_data/<string:stock_name>')

api.add_resource(_ReadStockGraph, '/stock_graph/<string:stock_name>')


def train_stock_prediction_model(stock_name):
    end_date = datetime.now()
    start_date = end_date - pd.Timedelta(days=4856)

    df = yf.download(stock_name, start=start_date, end=end_date)

    # preprocess the data
    df['Close'] = df['Close'].pct_change()  # calc daily returns
    df = df.dropna()

    X = df[['Open', 'High', 'Low', 'Close']].values
    y = (df['Close'] > 0).astype(int)  # 1 if price increaseds 0 if decreased

    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(4,)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy',
                  etrics=['accuracy'])
    model.fit(X, y, epochs=10)

    return model


def get_stock_recommendation(stock_name, model):
    end_date = datetime.now()
    start_date = end_date - pd.Timedelta(days=1)

    df = yf.download(stock_name, start=start_date, end=end_date)

    if df.empty:
        return {'error': 'No data found for the provided stock ticker.'}, 404

    df['Close'] = df['Close'].pct_change().iloc[-1]

    X = df[['Open', 'High', 'Low', 'Close']].values
    prediction = model.predict(X)

    recommendation = 'Buy' if prediction > 0.5 else 'Don\'t Buy'
    reason = f'This is based on the current data'

    return {'recommendation': recommendation, 'reason': reason}


class _AnalyzeStock(Resource):
    def get(self, stock_name):
        try:
            model = train_stock_prediction_model(stock_name)
            recommendation = get_stock_recommendation(stock_name, model)
            return jsonify(recommendation)
        except Exception as e:
            return jsonify({'error': str(e)}), 500


api.add_resource(_AnalyzeStock, '/analyze/<string:stock_name>')


## Explanation
This is a Python code that sets up a Flask application to provide a RESTful API for fetching stock data and performing analysis on it. Here's an explanation of the code:

1. **Imported Libraries**: The code starts by importing several Python libraries including Flask, pandas, yfinance, plotly, datetime, flask_restful, numpy, flask_cors, and TensorFlow.

2. **Blueprint Setup**: Flask Blueprint is created for the stock API.

3. **Cross-Origin Resource Sharing (CORS)**: CORS is enabled to allow cross-origin requests.

4. **`get_stock_graph` Function**: This function fetches historical stock data using the Yahoo Finance API (`yfinance`) and creates a candlestick chart using the Plotly library.

5. **`_ReadStockGraph` Class**: This class is a Flask-RESTful resource that provides an endpoint for retrieving historical stock data in the form of a candlestick chart.

6. **`_GetLatestStockData` Class**: This class provides an endpoint for fetching the latest stock data for a specific stock.

7. **`train_stock_prediction_model` Function**: This function fetches historical stock data, preprocesses it, and trains a simple neural network model using TensorFlow/Keras to predict whether the stock price will increase or decrease.

8. **`get_stock_recommendation` Function**: This function uses the trained model to make a recommendation on whether to buy or not buy a particular stock based on the latest data.

9. **`_AnalyzeStock` Class**: This class provides an endpoint for analyzing a specific stock by training a model and providing a recommendation based on the latest data.

10. **API Endpoints**: The Flask-RESTful API is set up with the necessary routes for fetching historical data, getting the latest data, and performing stock analysis.

Overall, this code sets up a Flask application that can be used to analyze stock data and provide recommendations on whether to buy or not buy a specific stock based on historical and latest data.


## Why This is Important
Certainly! The imports in the provided code are important for various functionalities required in the application that displays stock graphs and provides stock recommendations. Here's an explanation of each import:

1. **Flask**: Flask is a popular web framework in Python that is used to build web applications. It is essential for creating the web application and defining the routes for displaying stock data and recommendations.

2. **Blueprint, jsonify, request**: These are necessary for creating modular and reusable components within the Flask application. They help in defining specific routes and handling HTTP requests and responses, which are crucial for fetching and displaying stock data.

3. **pandas**: Pandas is a powerful data manipulation library in Python. It is used for data analysis and manipulation, making it essential for handling and processing stock data before displaying it.

4. **yfinance**: This library allows access to Yahoo Finance's API, enabling the retrieval of stock market data. It is crucial for fetching stock data and using it to display graphs and make recommendations.

5. **plotly.graph_objs**: Plotly is a visualization library that provides interactive and high-quality graphs. With `graph_objs`, the code can create visually appealing and informative stock graphs for better visualization of the data.

6. **datetime**: This library is important for handling date and time-related operations. It is necessary for specifying the date range when fetching historical stock data and for various other time-related calculations.

7. **Flask-Restful**: Flask-Restful is an extension for Flask that simplifies the creation of RESTful APIs. It makes it easier to define resources and their corresponding HTTP methods, which are crucial for creating endpoints for the stock data and recommendations.

8. **numpy**: NumPy is a fundamental package for scientific computing in Python. It provides support for large, multi-dimensional arrays and matrices, which might be useful for various calculations and data manipulation tasks when dealing with stock data.

9. **flask_cors**: Flask-CORS is a Flask extension for handling Cross-Origin Resource Sharing (CORS), which is crucial for allowing the application to make requests to a different domain. This is important when the application needs to fetch data from an external source, such as a stock market API.

10. **TensorFlow and Keras**: These libraries are essential for creating and training machine learning models. In this case, they are used to train a neural network model for predicting stock prices. TensorFlow is a powerful open-source machine learning framework, and Keras is an API that serves as an interface for TensorFlow, making it easier to build and train deep learning models.

These imports collectively provide the necessary tools and functionality to create a web application that can display stock graphs and provide recommendations based on historical and latest stock data.


## Student Lessons Organization:
1. Introduction to Python basics
2. Data types and data structures in Python
3. Control structures: loops and conditionals
4. Functions and modules in Python
5. Object-oriented programming in Python
6. File handling and input/output operations
7. Advanced topics: libraries and frameworks

## Usage of Blog as a Study Guide:
- Regular updates with new content and examples
- Categorization of topics for easy navigation
- Integration of external resources and links for further reading
- Regular self-assessment quizzes and practice exercises

## Pseudo Code Notes:
### Example 1: Pseudo Code for a simple sorting algorithm
```
Start with an unsorted list
Set a flag to check if the list is already sorted
Repeat until the flag is True:
    Set the flag to True
    For each element in the list:
        If the current element is greater than the next element:
            Swap the elements
            Set the flag to False
```

### Example 2: Pseudo Code for a basic iterative function
```
Set a variable n to a desired value
Set a variable result to 1
Repeat n times:
    Multiply result by n
    Decrement n by 1
Return the result
```

## College Board Quiz Notes:
- Researched the differences between '==' and 'is' in Python for question 3
- Looked up the syntax for list comprehension for question 5
- Revised the concept of dictionary methods for question 7
- Corrected misunderstanding of tuple unpacking for question 9

## Trimester 1 Reflection:
### Memories and Learnings:
- Enjoyed learning the fundamentals of Python programming
- Grasped the importance of structured programming techniques
- Found object-oriented programming concepts intriguing and challenging

### Positive Accomplishments:
- Successfully completed all the programming assignments
- Received positive feedback on the class project
- Improved problem-solving skills through coding challenges

### Intentions for Future Trimesters:
- Deepen understanding of data science applications in Python
- Explore more advanced topics in algorithms and data structures
- Enhance proficiency in using Python libraries for web development

### Opportunities for Growth:
- Strengthening debugging skills for efficient code troubleshooting
- Engaging in more collaborative programming projects with peers
- Seeking mentorship for guidance on complex programming concepts


## Challenges we as a team faced
#1: My commits stopped working for a period of time and until we fixed the issue I was unable to commit anything. While we were focusing on fixing this, our temporary solution was to call to complete code and/or I would send my coded parts to someone else so they could commit the code, allowing it to show up.

#2: Unclear idea of the project. Though we all knew the project was about finance we all had different views of what it would do. For example at the beginning we were divided n whether it should have been an organizer for taxes for a person ort a stocks app. We finally sat down and discussed our fial goal and decided on focusing on stocks and how a beginner could learn about them

#3: Connecting frontend and backend. I believe this was an issue for majority of teams but we had trouble connecting our frontend and backend as well as working to fix up our frontend in a way that works accordingly to the backend. This allowed for all team members to have a deeper understanding of parts of the code they didn't do

Overall, communication was the thing that fixed our various problems and we were able to work as a team and understand our project as a group to prevent issues. 